In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from datetime import date
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
pd.options.display.float_format = '{:.5f}'.format # 숫자형 데이터 지수표현법에서 float로 변경

In [2]:
users = pd.read_csv('users_RFM.csv', encoding = 'utf-8')

In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544921 entries, 0 to 544920
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   customer_id  544921 non-null  int64
 1   recency      544921 non-null  int64
 2   frequency    544921 non-null  int64
 3   monetary     544921 non-null  int64
 4   R            544921 non-null  int64
 5   F            544921 non-null  int64
 6   M            544921 non-null  int64
dtypes: int64(7)
memory usage: 29.1 MB


### RFM 분석

#### R-F 시각화 및 분석

In [81]:
rf_matrix = users.groupby(['R','F']).agg({'customer_id' : 'count', 'monetary' : 'sum'}).reset_index()
rf_matrix['avg_sales'] = rf_matrix['monetary'] / rf_matrix['customer_id']

In [82]:
rf_pivot = rf_matrix.pivot(index='F', columns='R', values='customer_id').fillna(0).sort_index(ascending=False)
rf_pivot.style.background_gradient(cmap ='Blues', axis=None, low=0.2)

R,1,2,3,4,5
F,,,,,
5,70,972,3637,10669,29514
4,460,3404,8888,15958,20873
3,5228,17140,27598,32410,27192
2,35297,44054,43932,33718,19760
1,68010,43352,28335,16060,8390


In [83]:
rf_pivot = rf_matrix.pivot(index='F', columns='R', values='avg_sales').fillna(0).sort_index(ascending=False)
rf_pivot.style.background_gradient(cmap ='Blues', axis=None, low=0.2)

R,1,2,3,4,5
F,,,,,
5,677505.142857,872657.155350,805058.838878,849445.527885,1045352.602290
4,350022.321739,444837.627203,410733.411679,391528.753290,391200.847458
3,176293.688026,236815.814061,193113.012247,184586.413268,186406.852236
2,74906.690625,104741.511282,69292.810594,64805.759179,74987.260779
1,33774.273416,42114.509619,23483.708029,19925.583873,29701.796186


In [84]:
rf_pivot = rf_matrix.pivot(index='F', columns='R', values='monetary').fillna(0).sort_index(ascending=False) / sum(rf_matrix['monetary'])
rf_pivot.style.background_gradient(cmap ='Blues', axis=None, low=0.2)

R,1,2,3,4,5
F,,,,,
5,0.000455,0.008143,0.028109,0.087002,0.296185
4,0.001546,0.014537,0.035046,0.059981,0.078389
3,0.008848,0.038967,0.051164,0.057432,0.048660
2,0.025382,0.044297,0.029224,0.020977,0.014225
1,0.022051,0.017527,0.006388,0.003072,0.002392


#### RFM 분석

     a. 최고가치 유저 그룹 R5&F5
     b. 단기 이탈 가치 유저 그룹 R4&F2~5
     c. 장기 이탈 가치 유저 그룹 R2~3&F2~5
     d. 최근 구매 가치 그룹 R5&F1~4

In [47]:
# a. 최고가치 유저 그룹 R5F4~5
users[(users['R'] == 5) & (users['F'] >= 4)].groupby(['R', 'M']).agg({'customer_id' : 'count', 'monetary' : 'median'})

# 금액대에 따라 세분화 M3~4 / M5

customer_id      monetary
R M                           
5 1            3   42500.00000
  2           61   81426.00000
  3         9617  230695.00000
  4        24562  492113.50000
  5        16144 1224719.50000

In [72]:
# b. 단기 이탈 유저 그룹 R4&F2~5
users[(users['R'] == 4) & ((users['F'] >= 2) & (users['F'] <= 5))]\
        .groupby(['R', 'M']).agg({'customer_id' : 'count', 'monetary' : 'median'})

# M1~3 저가치 / M4~5 고가치

customer_id      monetary
R M                           
4 1        20577   30880.00000
  2        15131   67400.00000
  3        33674  162811.00000
  4        17929  447390.00000
  5         5444 1107330.00000

In [41]:
#c. 장기 이탈 가치 유저 그룹 R1~3&F3~5
users[(users['R'] <= 3) & ((users['F'] >= 3) & (users['F'] <= 5))]\
        .groupby(['M']).agg({'customer_id' : 'count', 'monetary' : 'median'})

# M1~3 저가치 유저 / M4~5 고가치 유저

,customer_id,monetary
M,,
1,3338,40084.00000
2,10583,71900.00000
3,36017,161316.00000
4,13757,419320.00000
5,3702,1151448.00000


In [50]:
#d. 최근 구매 가치 그룹 R5&F1~3
users[(users['R'] == 5) & (users['F'] <= 3)].groupby(['R', 'M']).agg({'customer_id' : 'count', 'monetary' : 'median'})

# R5F1~3 구매 독려

customer_id      monetary
R M                           
5 1        19043   24500.00000
  2        10404   68118.50000
  3        22110  147630.00000
  4         3328  380070.00000
  5          457 1077489.00000

#### RFM Segmentation

In [3]:
users['segment'] = 'NaN'

In [9]:
## Segmentation
# Champions
print("champions %:", round(len(users[(users['R'] == 5) & (users['F'] >= 4) & (users['M'] == 5)]['segment']) / len(users), 2))

# Loyal Customers
print("Loyal Customer %:", round(len(users[(users['R'] == 5) & (users['F'] >= 4) & ((users['M'] >= 3) & (users['M'] <= 4))]['segment']) / len(users), 2))

# Recent Customer
print("Recent Customer %:", round((len(users[(users['R'] == 5) & (users['F'] == 4) & (users['M'] < 3)]['segment']) + len(users[(users['R'] == 5) & (users['F'] < 4)]['segment'])) / len(users), 2))

# HV CNA
print("HV CNA %:", round(len(users[(users['R'] == 4) & (users['F'] >= 2) & (users['M'] >= 4)]['segment']) / len(users), 2))

# LV CNA
print("LV CNA %:", round(len(users[(users['R'] == 4) & (users['F'] >= 2) & (users['M'] < 4)]['segment']) / len(users), 2))

# About to Sleep
print("About to Sleep", round(len(users[(users['R'] <= 3) & (users['F'] >= 2) & (users['M'] >= 2)]['segment']) / len(users), 2))

# At Risk
print("At Risk %:", round(len(users[(users['R'] <= 3) & (users['F'] >= 3) & (users['M'] >= 4)]['segment']) / len(users), 2))

# Hibernating
print("Hibernating %:", round(len(users[(users['R'] <= 3) & (users['F'] == 2) & (users['M'] <= 3)]['segment']) / len(users), 2))

champions %: 0.03
Loyal Customer %: 0.06
Recent Customer %: 0.1
HV CNA %: 0.04
LV CNA %: 0.13
About to Sleep 0.21
At Risk %: 0.03
Hibernating %: 0.22


In [10]:
## Segmentation

# champions
users.loc[(users['R'] == 5) & (users['F'] >= 4) & (users['M'] == 5), 'segment'] = 'C'

# Loyal Customers
users.loc[(users['R'] == 5) & (users['F'] >= 4) & ((users['M'] >= 3) & (users['M'] <= 4)), 'segment'] = 'LC'

# Recent Customer
users.loc[(users['R'] == 5) & (users['F'] == 4) & (users['M'] < 3), 'segment'] = 'RC'
users.loc[(users['R'] == 5) & (users['F'] < 4) , 'segment'] = 'RC'

# HV CNA
users.loc[(users['R'] == 4) & (users['F'] >= 2) & (users['M'] >= 4), 'segment'] = 'H_CNA'

# LV CNA
users.loc[(users['R'] == 4) & (users['F'] >= 2) & (users['M'] < 4), 'segment'] = 'L_CNA'

# About To Sleep
users.loc[(users['R'] <= 3) & (users['F'] >= 3) & (users['M'] >= 2), 'segment'] = 'AS'

# At Rist
users.loc[(users['R'] <= 3) & (users['F'] >= 3) & (users['M'] >= 4), 'segment'] = 'AR'

# Hibernating
users.loc[(users['R'] <= 3) & (users['F'] == 2) & (users['M'] <= 3), 'segment'] = 'H'

In [11]:
# segmentation별 비중
users.segment.value_counts() / len(users) * 100

H       22.46766
L_CNA   12.73249
RC      10.16771
AS       9.32062
LC       6.27229
H_CNA    4.28925
AR       3.20395
C        2.96263
Name: segment, dtype: float64